In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV
import shap
import lightgbm as lgb 

pd.set_option('max_colwidth', 80)

In [11]:
df = pd.read_csv('PipesWithYearlyBreaks100ft.csv') 
df_breaks = pd.read_csv('SeattleBreaksAndNearPipes.csv')
print(df.shape)
print(df_breaks.shape)

(1, 76)
(1, 8)


In [3]:
df = df.rename( columns=
    {'Join_Count_1': 'Breaks_nearby_2009', 
     'Join_Count_12': 'Breaks_nearby_2010',
       'Join_Count_12_13': 'Breaks_nearby_2011', 
       'Join_Count_12_13_14': 'Breaks_nearby_2012', 
       'Join_Count_12_13_14_15': 'Breaks_nearby_2013',
       'Join_Count_12_13_14_15_16': 'Breaks_nearby_2014', 
       'Join_Count_12_13_14_15_16_17': 'Breaks_nearby_2015',
       'Join_Count_12_13_14_15_16_17_18': 'Breaks_nearby_2016', 
       'Join_Count_12_13_14_15_16_17_18_19': 'Breaks_nearby_2017',
       'Join_Count_12_13_14_15_16_17_18_19_20': 'Breaks_nearby_2018',
       'Join_Count_12_13_14_15_16_17_18_19_20_21': 'Breaks_nearby_2019',
       'Join_Count_12_13_14_15_16_17_18_19_20_21_22': 'Breaks_nearby_2020',
       
    })

In [4]:
print(df.columns)

Index(['OBJECTID_1', 'Join_Count', 'TARGET_FID', 'Breaks_nearby_2009',
       'TARGET_FID_1', 'MNL_FEA_KE', 'MNL_FEAT_1', 'MNL_OWNE_1', 'MNL_MATE_1',
       'MNL_LENGTH', 'MNL_INSTAL', 'Width', 'NEAR_DIST', 'NEAR_X', 'NEAR_Y',
       'FID_SeaSoilClip', 'MUSYM', 'muname', 'OBJECTID', 'ARTCLASS', 'COMPKEY',
       'UNITID', 'UNITID2', 'UNITIDSORT', 'UNITDESC', 'STNAME_ORD', 'XSTRLO',
       'XSTRHI', 'ARTDESCRIP', 'OWNER', 'STATUS', 'BLOCKNBR', 'SPEEDLIMIT',
       'SEGDIR', 'ONEWAY', 'ONEWAYDIR', 'FLOW', 'SEGLENGTH', 'SURFACEWID',
       'SURFACETYP', 'SURFACET_1', 'INTRLO', 'DIRLO', 'INTKEYLO', 'INTRHI',
       'DIRHI', 'NATIONHWYS', 'STREETTYPE', 'PVMTCONDIN', 'PVMTCOND_1',
       'TRANCLASS', 'TRANDESCRI', 'SLOPE_PCT', 'PVMTCATEGO', 'PARKBOULEV',
       'SHAPE_Leng', 'Shape_Length_1', 'WONUM', 'ASSETNUM', 'Size', 'Long',
       'Lat', 'DATE', 'Shape_Length', 'Breaks_nearby_2010',
       'Breaks_nearby_2011', 'Breaks_nearby_2012', 'Breaks_nearby_2013',
       'Breaks_nearby_2014', 'Br

In [7]:
columns = [
       'MNL_FEAT_1', 
       'MNL_MATE_1', 
       'MNL_PIPE_1',
       'MNL_LENGTH',
       'MNL_INSTAL',
       'Size', 
       'MUSYM',
       'ARTCLASS',
       'SURFACEWID',
       'SPEEDLIMIT',
       'Width'
       #'Long', 
       #'Lat',
       'DATE', 
       'Breaks_nearby_2009', 'Breaks_nearby_2010',
       'Breaks_nearby_2011', 'Breaks_nearby_2012', 'Breaks_nearby_2013',
       'Breaks_nearby_2014', 'Breaks_nearby_2015', 'Breaks_nearby_2016',
       'Breaks_nearby_2017', 'Breaks_nearby_2018', 'Breaks_nearby_2019',
       'Breaks_nearby_2020']

In [36]:
ddff = df[df['DATE'].notna()]
print(len(ddff['DATE']))
print(len(ddff['DATE'].unique()))
print('duplicate breaks: ', len(ddff['DATE']) - len(ddff['DATE'].unique()))

4806
2380
duplicate breaks:  2426


In [37]:
# This line removes duplicate breaks, based on the date column. 
df_clean = df[(~df['DATE'].duplicated() | df['DATE'].isnull())]
df_clean = df_clean[columns]
df_clean.head()

,MNL_FEAT_1,MNL_MATE_1,MNL_PIPE_1,MNL_LENGTH,MNL_INSTAL,Size,DATE,Breaks_nearby_2009,Breaks_nearby_2010,Breaks_nearby_2011,Breaks_nearby_2012,Breaks_nearby_2013,Breaks_nearby_2014,Breaks_nearby_2015,Breaks_nearby_2016,Breaks_nearby_2017,Breaks_nearby_2018,Breaks_nearby_2019,Breaks_nearby_2020
0,Stub,Concrete,Circular,6.06,1/1/1960 0:00:00,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
1,Stub,Concrete,Circular,7.36,1/1/1959 0:00:00,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
2,Mainline,Concrete,Circular,43.75,1/1/1966 0:00:00,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
3,Mainline,Reinforced Concrete Pipe,Circular,16.80,1/1/1960 0:00:00,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
4,Mainline,Reinforced Concrete Pipe,Circular,38.67,1/1/1960 0:00:00,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
ffdd = df_clean[df_clean['DATE'].notna()]
print(len(ffdd['DATE']))
print(len(ffdd['DATE'].unique()))
print('duplicate breaks: ', len(ffdd['DATE']) - len(ffdd['DATE'].unique()))

2380
2380
duplicate breaks:  0


In [34]:
# select only years
df_clean['MNL_INSTAL'] = pd.to_datetime(df_clean['MNL_INSTAL'], format='%m/%d/%Y %H:%M:%S')
df_clean['MNL_INSTAL'] = df_clean['MNL_INSTAL'].map(lambda x: x.year)

df_clean['DATE'] = pd.to_datetime(df_clean['DATE'], format='%m/%d/%Y %H:%M:%S')
df_clean['DATE'] = df_clean['DATE'].map(lambda x: x.year)

df_clean.head()

,MNL_FEAT_1,MNL_PRBL_1,MNL_USE__1,MNL_MATE_1,MNL_PIPE_1,MNL_LENGTH,MNL_INSTAL,Size,Long,Lat,...,Breaks_nearby_2011,Breaks_nearby_2012,Breaks_nearby_2013,Breaks_nearby_2014,Breaks_nearby_2015,Breaks_nearby_2016,Breaks_nearby_2017,Breaks_nearby_2018,Breaks_nearby_2019,Breaks_nearby_2020
0,Stub,Sanitary,Sanitary,Concrete,Circular,6.06,1960,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,Stub,Sanitary,Sanitary,Concrete,Circular,7.36,1959,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,Mainline,Sanitary,Sanitary,Concrete,Circular,43.75,1966,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,Mainline,Sanitary,Sanitary,Reinforced Concrete Pipe,Circular,16.80,1960,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Mainline,Sanitary,Sanitary,Reinforced Concrete Pipe,Circular,38.67,1960,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# filter out NaN from DATE col
breaks_df = df_clean[df_clean['DATE'].notna()]
# set size column to width column and drop size 
breaks_df['Pipe_widths_Width'] = breaks_df['Size']
breaks_df = breaks_df.drop('Size', axis=1)
# breaks_df.head()

In [38]:
pseudo_df = df_clean
# pseudo_df.head()

# fill Nan with 0 (idk why, but it just made it work FOR NOW)
# if date is not 0 (NaN), make width = size 
# then for all df, drop size column

pseudo_df['DATE'] = pseudo_df['DATE'].fillna(0)
pseudo_df.loc[pseudo_df['DATE'] != 0, ['Pipe_wid_1']] = pseudo_df['Size']
pseudo_df = pseudo_df.drop('Size', axis=1)
# pseudo_df.head()

# change 'Width' values to numbers instead of strings (for dummy prep)
pseudo_df['Pipe_wid_1'] = pd.to_numeric(pseudo_df['Pipe_wid_1'], errors='coerce')
#pseudo_df['Pipe_widths_Width'] = pseudo_df['Width'].map(lambda x: float(x), na_action='ignore')
# pseudo_df['Width'].unique()

# Assign binary variables:
# [Create new column] If pipe has a broken date --> broken pipes = 0, non-broken pipes = 1
pseudo_df['TARGET'] = pseudo_df['DATE'].apply(lambda x: 0 if x == 0 else 1)

In [61]:
# Create dummy variables
dummy_df = pd.get_dummies(pseudo_df)
aaaa = (dummy_df.columns)
for col in aaaa:
    print(col)

DWW_Main_4
DWW_Main_5
Pipe_wid_1
MUSYM
ARTCLASS
BLOCKNBR
SPEEDLIMIT
SURFACEWID
SLOPE_PCT
NEAR_DIST
DATE
Shape_Leng
Join_Cou_2
Join_Cou_3
Join_Cou_4
Join_Cou_5
Join_Cou_6
Join_Cou_7
Join_Cou_8
Join_Cou_9
Join_Co_10
Join_Co_11
Join_Co_12
Join_Co_13
TARGET
DWW_Main_3_ 
DWW_Main_3_Acrylonitrile Butadiene Styrene
DWW_Main_3_Asbestos Cement
DWW_Main_3_Brick
DWW_Main_3_Cast Iron Pipe
DWW_Main_3_Concrete
DWW_Main_3_Corrugated Flexible Plastic
DWW_Main_3_Corrugated Metal Pipe
DWW_Main_3_Corrugated Rigid Plastic
DWW_Main_3_Ductile Iron Pipe
DWW_Main_3_High Density Polyethylene
DWW_Main_3_Other
DWW_Main_3_Polyvinyl Chloride
DWW_Main_3_Reinforced Concrete Box
DWW_Main_3_Reinforced Concrete Pipe
DWW_Main_3_Steel
DWW_Main_3_Unknown
DWW_Main_3_Vitrified Clay
DWW_Main_3_Wood Stave Pipe
SURFACETYP_ 
SURFACETYP_AC
SURFACETYP_AC/AC
SURFACETYP_AC/PCC
SURFACETYP_GRAVEL
SURFACETYP_PCC
SURFACETYP_ST


In [164]:
def get_data(df, start, end):
    """
    Takes in df and filters depending on timeframe (start and end years).
    Returns subset of data for training in timeframe.
    """
    # want to include where there is NOT a break year (those will be our non-broken positive examples --> 'DATE' == 0) - DATE col


    # want to include where break year is in time frame of what we want - DATE col
    filt_df = df[(df['DATE'] == 0 )| ((df['DATE'] >= start) & (df['DATE'] <= end))]

    # exclude installs AFTER time frame window - MNL_INSTAL col
    filt_df = filt_df[(filt_df['DWW_Main_5'] <= end)]

    # based on time window, calculate appropriate age of pipes (select beginning year of time frame --> ex: 2009, 2010, 2011
    #       subtract install year from 2009) - MNL_INSTAL col
    # -- will create negative numbers
    filt_df['AGE'] = start - filt_df['DWW_Main_5']

    print(df['Join_Cou_7'].sum())

    print(filt_df['Join_Co_10'].sum())
    
    return filt_df

In [165]:
train_data = get_data(dummy_df, 2014, 2016)
test_data = get_data(dummy_df, 2017, 2019)

112
0
112
122


In [110]:
# Calculate breaks for dif time segments for training data
train_data['breaks_one_yrs'] = train_data['Join_Cou_6']
# train_data['breaks_two_yrs'] = train_data['Join_Cou_6'] + train_data['Join_Cou_5']
# train_data['breaks_three_yrs'] = train_data['Join_Cou_6'] + train_data['Join_Cou_5'] + train_data['Join_Cou_4']
# train_data['breaks_four_yrs'] = train_data['Join_Cou_6'] + train_data['Join_Cou_5'] + train_data['Join_Cou_4'] + train_data['Join_Cou_3']
# train_data['breaks_five_yrs'] = train_data['Join_Cou_6'] + train_data['Join_Cou_5'] + train_data['Join_Cou_4'] + train_data['Join_Cou_3'] + train_data['Join_Cou_2']

In [106]:
# Calculate breaks for dif time segments for testing data
test_data['breaks_one_yrs'] = test_data['Join_Cou_9']
test_data['breaks_two_yrs'] = test_data['Join_Cou_9'] + test_data['Join_Cou_8']
test_data['breaks_three_yrs'] = test_data['Join_Cou_9'] + test_data['Join_Cou_8'] + test_data['Join_Cou_7']
test_data['breaks_four_yrs'] = test_data['Join_Cou_9'] + test_data['Join_Cou_8'] + test_data['Join_Cou_7'] + test_data['Join_Cou_6']
test_data['breaks_five_yrs'] = test_data['Join_Cou_9'] + test_data['Join_Cou_8'] + test_data['Join_Cou_7'] + test_data['Join_Cou_6'] + test_data['Join_Cou_5']

In [113]:
dummy_df['Join_Cou_6'].sum()

132

In [56]:
 # Calculate # of breaks for different time segments
# Joint_Cou_2 until Joint_Cou_13 goes from 2009 (Cou_2)-2020 (Cou_13).
yrs = np.arange(2009, 2021, 1)

joins_feature_list = [
    'Breaks_nearby_2009', 'Breaks_nearby_2010',
       'Breaks_nearby_2011', 'Breaks_nearby_2012', 'Breaks_nearby_2013',
       'Breaks_nearby_2014', 'Breaks_nearby_2015', 'Breaks_nearby_2016',
       'Breaks_nearby_2017', 'Breaks_nearby_2018', 'Breaks_nearby_2019',
       'Breaks_nearby_2020'
]


In [173]:
for col in joins_feature_list:
    print(col + ': ' + str(dummy_df[col].sum()))

Join_Cou_2: 90
Join_Cou_3: 42
Join_Cou_4: 49
Join_Cou_5: 109
Join_Cou_6: 132
Join_Cou_7: 112
Join_Cou_8: 135
Join_Cou_9: 126
Join_Co_10: 122
Join_Co_11: 112
Join_Co_12: 110
Join_Co_13: 82


In [54]:
# trains
# tests
# move over
# rinse & repeat until 2019
def split_df(df):
    """
    Takes in a dataframe and creatures a feature dataset and a tagrt datas
    """    
    df = df.dropna()
    feature_df = df.drop(['TARGET', 'DATE'], axis=1)
    target_df = df[['TARGET']]

    return feature_df, target_df

def train_v1(df, model):
    """
    Takes in a dataframe of interest and a model, and trains the model.
    Model likely needs to be one imported from sklearn so it is able to
    handle dataframes as input data
    """
    feature, target = split_df(df)
    clf = model #clf is for for classification
    clf.fit(feature, target)

    return clf

In [53]:
def main_function(dummy_df, start, end, model):
    """
    Takes in dummy dataframe and runs all functions based on given year ranges.
    Prints out year and accuracy per time range (3 years ranges)
    """

    full_results = {} # Dictionary for storing all the models and their data

    for i in range(start, end - 4):

        

        start_train = i
        end_train = i + 2
        df = get_data(dummy_df, start_train, end_train)
        feature_train, target_train = split_df(df)
        clf = train_v1(df, model)

        training_pred = clf.predict(feature_train)
        #print(start_train, "-", end_train, ": ")#, accuracy_score(y_true=target, y_pred=training_pred)) 

        # Test
        start_test = i + 3
        end_test = start_test + 2
        test = get_data(dummy_df, start_test, end_test)
        feature_test, target_test = split_df(test)
        testing_pred = clf.predict(feature_test)
        #testing_proba_preds = clf.predict_proba(feature_test)[::,1]
        #print(start_test, "-", end_test, ": ", accuracy_score(y_true=target_test, y_pred=testing_pred))
        
        print(start_train, "-", end_test, ": ")#, accuracy_score(y_true=target, y_pred=training_pred)) 


        # Print out break and non break accuracy's rather total full accuracy
        cm = confusion_matrix(y_true=target_test, y_pred=testing_pred)
        # nonbreak_acc = cm[0,0] / (cm[0,0] + cm[0,1])
        # print('Non-break accuracy: ', nonbreak_acc)
        # break_acc = cm[1,1] / (cm[1, 0] + cm[1,1])
        # print('Break accuracy: ', break_acc)

        #print out metrics
        # precision, recall, f1, support = precision_recall_fscore_support(y_true=target_test, y_pred=testing_pred)
        # print('Precision: ', precision)
        # print('Recall: ', recall)
        # print('F1 Score: ', f1)
        # print('Support: ', support)

        precision = precision_score(y_true=target_test, y_pred=testing_pred)
        recall = recall_score(y_true=target_test, y_pred=testing_pred)
        bal_acc = balanced_accuracy_score(y_true=target_test, y_pred=testing_pred)
        print('precision: ', precision)
        print('recall: ', recall)
        print('Balanced accuracy: ', bal_acc)
        
        print()

        model_results = {
            'model': clf,
            'start_year': start_test,
            'end_year': end_test,
            'test_df': feature_test, # For eventual SHAP calculations
            'target_values': target_test,
            'test_predictions': testing_pred,
            'confusion_matrix': cm

            }

        full_results['model_' + str(start_test) + '-' + str(end_test)] = model_results
    
    return full_results